In [ ]:
#%matplotlib inline
#from matplotlib import style
#style.use('fivethirtyeight')

#imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine,text, inspect, func

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Create engine using the `hawaii.sqlite` database file
engine = create_engine("sqlite:///Resources/hawaii.sqlite")


In [ ]:
# Create the inspector and connect it to the engine
inspector_gadget = inspect(engine)

# Collect the names of tables within the database
tables = inspector_gadget.get_table_names()

# print metadata for each table
for table in tables:
    print(table)
    print("-----------")
    
    # get columns
    columns = inspector_gadget.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])

    print()

In [ ]:
# Declare a Base using `automap_base()`
Base = automap_base()
# Use the Base class to reflect the database tables
Base.prepare(autoload_with=engine)
print(list(Base.classes))

In [ ]:
# Print all of the classes mapped to the Base
Base.classes.keys()

In [ ]:
# Assign the measurement class to a variable called `Measurement` and
# the station class to a variable called `Station`
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create a session
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
query="SELECT max(date) from Measurement"
with engine.connect() as conn:
    print(conn.execute(text(query)).fetchall())
    


In [ ]:
most_recent_date = session.query(func.max(Measurement.date)).first()
most_recent_date

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results.
# Starting from the most recent data point in the database.

query = """
    SELECT
        date,
        station,
        prcp
        
    FROM
        Measurement
    WHERE
        date >= '2016-08-23' AND date <= '2017-07-23'
        
    ORDER BY
        date asc;
"""

df1 = pd.read_sql(text(query), con=engine)
df1.info()


In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results.
# Starting from the most recent data point in the database.

# Calculate the date one year from the last date in data set.
prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)

# Perform a query to retrieve the data and precipitation scores
results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= prev_year)

# Save the query results as a Pandas DataFrame. Explicitly set the column names
df = pd.DataFrame(results, columns =['Date', 'Precipitation'])

# Sort the dataframe by date
df = df.sort_values("Date")

# Use Pandas Plotting with Matplotlib to plot the data
df.plot(x = 'Date', y = 'Precipitation' ,color="green",rot=90)
plt.xlabel("Date")
plt.ylabel("Inches")

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
df.describe()


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the dataset
no_of_stations = session.query(func.count(Station.station)).all()
no_of_stations

In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.
query= """
    SELECT station, 
    count(*) as num_rows
    from Measurement 
    group by station
    order by num_rows desc;
"""
df=pd.read_sql(text(query), con=engine)
df.head()

In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.
results= session.query(Measurement.station, func.count(Measurement.date)).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.date).desc()).\
    all()
#Save the query results as a pandas DataFrame. Explicitly set the column names
df2=pd.DataFrame(results, columns=["Station","Observations"])
df2.head()



In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
query= """
    SELECT min(tobs) as min_temp,
           max(tobs) as max_temp,
           avg(tobs) as avg_temp
    FROM
        Measurement
    WHERE station ='USC00519281';
"""
df_tobs=pd.read_sql(text(query), con=engine)
df_tobs.head()

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
results=session.query(func.min(Measurement.tobs),func.max(Measurement.tobs),func.Avg(Measurement.tobs)).\
    filter(Measurement.station == 'USC00519281' ).\
    all()

#Save the query results as a pandas DataFrame. Explicitly set the column names
df_temp=pd.DataFrame(results, columns=["min_temp","max_temp","avg_temp"])
df_temp.head()



In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
# Calculate the date one year from the last date in data set.
prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)

# Perform a query to retrieve the data and precipitation scores
results = session.query(Measurement.date, Measurement.station,Measurement.tobs).\
    filter(Measurement.date >= prev_year).\
    filter(Measurement.station == 'USC00519281').\
    all()

# Save the query results as a Pandas DataFrame. Explicitly set the column names
df4 = pd.DataFrame(results, columns =['Date', 'Station','TOBS'])

# Sort the dataframe by date
df4 = df4.sort_values("Date")

df4.head()


In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
plt.figure(figsize=(10,6))
plt.hist(df4.TOBS,color="green")
plt.xlabel("Temperature")
plt.ylabel("Frequency")
plt.legend("tobs")
plt.title("Histogram of Temperature for USC00519281")
plt.show()


# Close Session

In [ ]:
# Close Session
session.close()
